In [25]:
import pandas as pd
import folium
import googlemaps
from datetime import datetime
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist
from config import API_KEY

api_key= API_KEY

# Load destinations from a text file
def load_destinations(file_path):
    destinations = []
    with open(file_path, 'r') as file:
        for line in file:
            destination = line.strip()
            if destination:
                destinations.append(destination)
    return destinations

def get_lat_lon(destination, api_key):
    """
    Get latitude and longitude of a destination using geocoding.
    geocode should be the cheapest here
    """
    gmaps = googlemaps.Client(key=api_key)
    try:
        result = gmaps.geocode(destination)
        lat = result[0]["geometry"]["location"]["lat"]
        lon = result[0]["geometry"]["location"]["lng"]
        return lat, lon
    except googlemaps.exceptions.ApiError as e:
        print("Error occurred while retrieving geocode: {}".format(e))
        return None, None
    except:
        return None, None

def add_lat_lon_to_dataframe(destinations, api_key=api_key):
    """
    Loop through a list of destinations, get their latitude and longitude using geocoding,
    and append the latitude and longitude to a new column in a pandas dataframe.
    """
    # Create an empty dataframe to store the results
    df = pd.DataFrame(columns=['Destination', 'Latitude', 'Longitude'])
    
    # Loop through the list of destinations and add the latitude and longitude to the dataframe
    for destination in destinations:
        lat, lon = get_lat_lon(destination, api_key)
        df = df.append({'Destination': destination, 'Latitude': lat, 'Longitude': lon}, ignore_index=True)
    
    return df


In [43]:
def generate_shortest_path(destinations, api_key=api_key):

    df = add_lat_lon_to_dataframe(destinations, api_key)
    return df

In [117]:
destination = ["Kwun Tong", "Kowloon Tong", "Hang Hau", "Yuen Long","Kwai Hing"]

In [118]:
coor_df = generate_shortest_path(destination)


C:\Users\maria\AppData\Local\Temp\ipykernel_12956\292304892.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Destination': destination, 'Latitude': lat, 'Longitude': lon}, ignore_index=True)
C:\Users\maria\AppData\Local\Temp\ipykernel_12956\292304892.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Destination': destination, 'Latitude': lat, 'Longitude': lon}, ignore_index=True)
C:\Users\maria\AppData\Local\Temp\ipykernel_12956\292304892.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Destination': destination, 'Latitude': lat, 'Longitude': lon}, ignore_index=True)
C:\Users\maria\AppData\Local\Temp\ipykernel_12956\292304892.py:50: FutureWarning: The frame.appen

In [119]:
coordinate_array = coor_df.iloc[:,1:].values
coordinate_array

array([[ 22.3103686, 114.2227035],
       [ 22.3369319, 114.1763367],
       [ 22.3172623, 114.2672828],
       [ 22.4445376, 114.0222076],
       [ 22.3630576, 114.13121  ]])

In [135]:

coor_df["Destination"].value

AttributeError: 'Series' object has no attribute 'value'

In [133]:
"edward is dumb" + str(["hong kong", "eng","ita"])

"edward is dumb['hong kong', 'eng', 'ita']"

In [97]:
distances

array([[1.05504895, 2.89279768, 1.20847651, 2.05545421]])

In [122]:
import numpy as np
from scipy.spatial.distance import cdist

# create a copy of the coordinate_array
remaining_location = coordinate_array.copy()
itinerary_list = []

# choose an arbitrary starting index
start_index = 2

while len(remaining_location) > 0:
    start = remaining_location[start_index]

    itinerary_list.append(start)

    # delete the already visited location
    remaining_location = np.delete(remaining_location, start_index, axis=0)

    if len(remaining_location) > 0:
        distances = cdist([start], remaining_location)
        closest_index = np.argmin(distances)
        start_index = closest_index
    else:
        break

itinerary_list = np.array(itinerary_list)

final_itinerary = pd.DataFrame()
for iters in range(len(itinerary_list)):
    sorted_temp_df = coor_df.query("(Latitude =={}) & (Longitude == {})".format(itinerary_list[iters][0],itinerary_list[iters][1]))
    final_itinerary = pd.concat([final_itinerary, sorted_temp_df])
final_itinerary.reset_index(inplace=True, drop=True)

[[ 22.3172623 114.2672828]
 [ 22.3103686 114.2227035]
 [ 22.3369319 114.1763367]
 [ 22.3630576 114.13121  ]
 [ 22.4445376 114.0222076]]


In [123]:
itinerary_list

array([[ 22.3172623, 114.2672828],
       [ 22.3103686, 114.2227035],
       [ 22.3369319, 114.1763367],
       [ 22.3630576, 114.13121  ],
       [ 22.4445376, 114.0222076]])

In [131]:
final_itinerary

,Destination,Latitude,Longitude
0,Hang Hau,22.317262,114.267283
1,Kwun Tong,22.310369,114.222703
2,Kowloon Tong,22.336932,114.176337
3,Kwai Hing,22.363058,114.131210
4,Yuen Long,22.444538,114.022208


In [85]:
np.argmin(distances)

0

In [42]:
# example from chat

# Define a point A
A = np.array([1.5, 1.5])

# Calculate the distances between A and all the other points
distances = cdist([A], points)

# Get the index of the closest point
closest_index = np.argmin(distances)

# Get the closest point
closest_point = points[closest_index]

# Calculate the distances between the closest point and all the other points
distances = cdist([closest_point], points)

# Sort the distances and choose the smallest non-zero distance
sorted_distances = np.sort(distances)
next_closest_distance = sorted_distances[sorted_distances > 0][0]

array([51.5072178, -0.1275862])

In [24]:
add_lat_lon_to_dataframe(destination)

TypeError: get_lat_lon() takes 1 positional argument but 2 were given

In [116]:
coor_df

,Destination,Latitude,Longitude
0,Birmingham,52.486243,-1.890401
1,London,51.507218,-0.127586
2,Manchester,53.480759,-2.242631
3,Nottingham,52.954022,-1.154989
4,Bristol,51.454513,-2.587910
